In [ ]:
%git clone https://github.com/TUD-STKS/SECAI-Summer-School.git
%cd SECAI-Summer-School
%pip install --upgrade .[notebook]

# Introduction to Machine Learning

- Teilnehmende werden fast kein Python kennen
- Google Colab
- Einführung in Python kurz halten -> evtl. zumindest kurz erklären, was eine Funktion ist
- Regression in sklearn
- Übergang zu PyTorch -> Motivieren, warum PyTorch für komplexere Modelle besser geignet ist
- Englisch!!!

In [ ]:
from joblib import dump, load
from sklearn.metrics import accuracy_score

from tqdm import tqdm
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision.transforms as transforms

from sklearn.decomposition import PCA
from sklearn.model_selection import RandomizedSearchCV, StratifiedShuffleSplit, PredefinedSplit
from sklearn.linear_model import RidgeClassifier, SGDClassifier
from scipy.stats import loguniform
from secai.torch_models import LinearRegression, MultiLayerPerceptron, ConvolutionalNeuralNetwork, LSTMModel, EarlyStopping

import medmnist
from medmnist import INFO, Evaluator

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [ ]:
sns.set_theme(context="talk")

cuda_available = torch.cuda.is_available()
device = torch.device('cuda') if cuda_available else torch.device('cpu')

# We first work on the 2D dataset called "PathMNIST"

The following cell give us some first information about this dataset. We are dealing with an image dataset containing RGB impage patches from hematoxylin & eosin stained histological images, obtained in different clinical centers.

Since we are dealing with RGB image patches, we have three different channels.

In total, there are nine different classes. Hence, we have a multi-class dataset, and each image is assigned to exactly one class.

The training and validation set (NCT-CRC-HE-100K) contain 100,000 patches, and the test set contains 7,180 image patches (CRC-VAL-HE-7K) from a different clinical center.

In [ ]:
data_flag = 'pathmnist'
# data_flag = 'breastmnist'

info = INFO[data_flag]
task = info['task']
labels = info['label']

n_channels = info['n_channels']
n_classes = len(info['label'])

info

We prepare to download the dataset (if it is not already downloaded) and instantiate

In [ ]:
download = True

## First, we read the raw MedMNIST data without any preprocessing

Since we want to dive deeper into the dataset, we do not apply any kind of preprocessing. We only make sure that the dataset class returns the data as `torch.Tensor`, such that we can easily analyze it further.

Note that we create two different datasets, one for training and one for test. This is something that we always need to keep in mind. Always split training and test data and make sure that no test data is used for training or parameter optimization.

In [ ]:
BATCH_SIZE = 1024
N_PIXELS = 28*28

validation_split = .2

# preprocessing
data_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[.5], std=[.5])
])

# load the data
DataClass = getattr(medmnist, info['python_class'])
train_dataset = DataClass(split='train', transform=data_transform,
                          download=download, as_rgb=True)
validation_dataset = DataClass(split='val', transform=data_transform,
                               download=download, as_rgb=True)
test_dataset = DataClass(split='test', transform=data_transform,
                         download=download, as_rgb=True)

train_loader = data.DataLoader(
    dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
validation_loader = data.DataLoader(
    dataset=validation_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = data.DataLoader(
    dataset=test_dataset, batch_size=2*BATCH_SIZE, shuffle=True)

In [ ]:
training_input = []
training_target = []
for data in tqdm(train_loader):
    training_input.append(data[0].numpy().reshape(-1, N_PIXELS))
    training_target.append(data[1].numpy().flatten())

training_df = pd.DataFrame(np.vstack(training_input), columns=[f"Pixel {k+1}" for k in range(N_PIXELS)])
training_df["Target"] = [labels[str(d)] for d in np.hstack(training_target)]
training_df["Numeric target"] = np.hstack(training_target)

In [ ]:
validation_input = []
validation_target = []
for data in tqdm(validation_loader):
    validation_input.append(data[0].numpy().reshape(-1, N_PIXELS))
    validation_target.append(data[1].numpy().flatten())

validation_df = pd.DataFrame(np.vstack(validation_input), columns=[f"Pixel {k+1}" for k in range(N_PIXELS)])
validation_df["Target"] = [labels[str(d)] for d in np.hstack(validation_target)]
validation_df["Numeric target"] = np.hstack(validation_target)

In [ ]:
training_df

In [ ]:
validation_df

## Visualization

Visualization is always a crucial part when getting started with a new dataset. Even when only looking on samples, we get a better idea of what is contained in the dataset.

Here, we observe severa interesting things:

- The pixel values (mean of the different RGB values) seem to be normalized, as we do not deal with integer values.
- The value ranges are different. In most of the images, the values seem to lie between 0.3 and 0.8, but not always.
- The histograms indicate that the pixel values overall are distributed reasonable.
- All pixels seem to carry information, as the distribution does not indicate that some pixel values have a small standard deviation.

All in all, this suggests that the pre-processing is simple in case of this dataset. We will simply shift each pixel value by 0.5 and 

In [ ]:
fig, axs = plt.subplots(3, 3, sharex="all", sharey="all")

for k in range(9):
    sns.heatmap(data=training_df.loc[k, [f"Pixel {k+1}" for k in range(N_PIXELS)]].values.astype(float).reshape(28, 28).T,
                ax=axs.flatten()[k], square=True, xticklabels=False, yticklabels=False)
    # axs.flatten()[k].set_title(training_df.loc[k, "Target"])

# [ax.set_xlabel("x") for ax in axs[-1, :]]
# [ax.set_ylabel("y") for ax in axs[:, 0]]
plt.tight_layout()

In [ ]:
fig, axs = plt.subplots(2, 1, sharex="all")

sns.histplot(data=training_df.loc[:, [f"Pixel {k+1}" for k in range(0, 5)]], ax=axs[0])
sns.boxplot(data=training_df.loc[:, [f"Pixel {k+1}" for k in range(0, 5)]], ax=axs[1], orient="h")

plt.tight_layout()
# plt.savefig("histogram_vs_boxplot.png", bbox_inches="tight")

In [ ]:
pca = PCA().fit(training_df.loc[:, [f"Pixel {k+1}" for k in range(N_PIXELS)]])

In [ ]:
fig, axs = plt.subplots()

sns.lineplot(x=np.arange(1, 785), y=np.cumsum(pca.explained_variance_ratio_), ax=axs)
axs.axhline(y=0.95, c="r")
axs.set_xlabel("Pixel k")
axs.set_ylabel("Accumulated explained variance")
axs.set_xlim((0, N_PIXELS+5))
axs.set_ylim((0.55, 1.01))
plt.tight_layout()
plt.savefig("pca_explained_variance_ratio.png", bbox_inches="tight")

In [ ]:
cv_training_df = pd.concat((training_df, validation_df))
test_fold = [-1] * len(training_df) + [1] * len(validation_df)

cv = PredefinedSplit(test_fold=test_fold)

In [ ]:
try:
    clf = load("results/sklearn_linear_model.joblib")
except FileNotFoundError:
    clf = RandomizedSearchCV(estimator=SGDClassifier(loss="log_loss"), n_iter=50,
                             n_jobs=-1, cv=cv, verbose=10,
                             param_distributions={"alpha": loguniform(a=1e-5, b=1e1)}).fit(
        cv_training_df.loc[:, [f"Pixel {k+1}" for k in range(N_PIXELS)]], y=cv_training_df.loc[:, "Numeric target"])
    dump(clf, "results/sklearn_linear_model.joblib")

In [ ]:
fig, axs = plt.subplots()

sns.lineplot(data=pd.DataFrame(clf.cv_results_), x="param_alpha", y="mean_test_score", ax=axs)
plt.xscale("log")
plt.tight_layout()

In [ ]:
clf.score(training_df.loc[:, [f"Pixel {k+1}" for k in range(N_PIXELS)]],
          y=training_df.loc[:, "Numeric target"])

In [ ]:
test_input = []
test_target = []
for data in tqdm(test_loader):
    test_input.append(data[0].numpy().reshape(-1, N_PIXELS))
    test_target.append(data[1].numpy().flatten())

In [ ]:
test_df = pd.DataFrame(np.vstack(test_input), columns=[f"Pixel {k+1}" for k in range(N_PIXELS)])
test_df["Target"] = [labels[str(d)] for d in np.hstack(test_target)]
test_df["Numeric target"] = np.hstack(test_target)

In [ ]:
clf.score(test_df.loc[:, [f"Pixel {k+1}" for k in range(N_PIXELS)]], y=test_df.loc[:, "Numeric target"])

## Then, we define a simple model for illustration, object function and optimizer that we use to classify.

In [ ]:
def train_model(model, patience, n_epochs, path, training_dataloader, validation_dataloader):
    # to track the training loss as the model trains
    training_losses = []
    # to track the validation loss as the model trains
    validation_losses = []
    # to track the average training loss per epoch as the model trains
    average_training_losses = []
    # to track the average validation loss per epoch as the model trains
    average_validation_losses = [] 
    # initialize the early_stopping object
    early_stopping = EarlyStopping(patience=patience, verbose=True, path=path)
    for epoch in range(1, n_epochs + 1):
        ###################
        # train the model #
        ###################
        model.train() # prep model for training
        for batch, (data, target) in enumerate(training_dataloader, 1):
            data = data.to(device)
            target = target.to(device)
            # clear the gradients of all optimized variables
            optimizer.zero_grad()
            # forward pass: compute predicted outputs by passing inputs to the model
            output = model(data)
            # calculate the loss
            target = target.squeeze().long()
            loss = criterion(output, target)
            # backward pass: compute gradient of the loss with respect to model parameters
            loss.backward()
            # perform a single optimization step (parameter update)
            optimizer.step()
            # record training loss
            training_losses.append(loss.item())
        ######################    
        # validate the model #
        ######################
        model.eval() # prep model for evaluation
        for data, target in validation_dataloader:
            data = data.to(device)
            target = target.to(device)
            # forward pass: compute predicted outputs by passing inputs to the model
            output = model(data)
            # calculate the loss
            target = target.squeeze().long()
            loss = criterion(output, target)
            # record validation loss
            validation_losses.append(loss.item())
        # print training/validation statistics 
        # calculate average loss over an epoch
        training_loss = np.average(training_losses)
        validation_loss = np.average(validation_losses)
        average_training_losses.append(training_loss)
        average_validation_losses.append(validation_loss)
        epoch_len = len(str(n_epochs))
        print_msg = (f'[{epoch:>{epoch_len}}/{n_epochs:>{epoch_len}}] '
                     f'train_loss: {training_loss:.5f} '
                     f'valid_loss: {validation_loss:.5f}')
        print(print_msg)
        # clear lists to track next epoch
        train_losses = []
        valid_losses = []
        # early_stopping needs the validation loss to check if it has decresed, 
        # and if it has, it will make a checkpoint of the current model
        early_stopping(validation_loss, model, optimizer, epoch)
        if early_stopping.early_stop:
            print("Early stopping")
            break
    return  model, optimizer, early_stopping.epoch, loss, average_training_losses, average_validation_losses

In [ ]:
model = LinearRegression(in_features=784, num_classes=n_classes)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=1e-2, weight_decay=1e-2)

In [ ]:
patience = 5
num_epochs = 500
model, optimizer, epoch, loss, average_training_losses, average_validation_losses = train_model(
    model, patience, num_epochs, "results/torch_linear_model.pt", train_loader, validation_loader)

[281/500] train_loss: 1.94192 valid_loss: 1.96417
Validation loss decreased (1.964174 --> 1.964172).  Saving model ...
[282/500] train_loss: 1.94191 valid_loss: 1.96417
Validation loss decreased (1.964172 --> 1.964165).  Saving model ...
[283/500] train_loss: 1.94190 valid_loss: 1.96416
Validation loss decreased (1.964165 --> 1.964158).  Saving model ...
[284/500] train_loss: 1.94189 valid_loss: 1.96416
EarlyStopping counter: 1 out of 5
[285/500] train_loss: 1.94188 valid_loss: 1.96415
Validation loss decreased (1.964158 --> 1.964153).  Saving model ...
[286/500] train_loss: 1.94187 valid_loss: 1.96415
Validation loss decreased (1.964153 --> 1.964145).  Saving model ...
[287/500] train_loss: 1.94186 valid_loss: 1.96414
Validation loss decreased (1.964145 --> 1.964140).  Saving model ...
[288/500] train_loss: 1.94185 valid_loss: 1.96414
Validation loss decreased (1.964140 --> 1.964139).  Saving model ...
[289/500] train_loss: 1.94185 valid_loss: 1.96413
Validation loss decreased (1.9641

[353/500] train_loss: 1.94139 valid_loss: 1.96386
Validation loss decreased (1.963869 --> 1.963863).  Saving model ...
[354/500] train_loss: 1.94138 valid_loss: 1.96386
Validation loss decreased (1.963863 --> 1.963859).  Saving model ...
[355/500] train_loss: 1.94137 valid_loss: 1.96385
Validation loss decreased (1.963859 --> 1.963854).  Saving model ...
[356/500] train_loss: 1.94137 valid_loss: 1.96385
Validation loss decreased (1.963854 --> 1.963848).  Saving model ...
[357/500] train_loss: 1.94136 valid_loss: 1.96384
Validation loss decreased (1.963848 --> 1.963841).  Saving model ...
[358/500] train_loss: 1.94136 valid_loss: 1.96384
Validation loss decreased (1.963841 --> 1.963838).  Saving model ...
[359/500] train_loss: 1.94135 valid_loss: 1.96383
Validation loss decreased (1.963838 --> 1.963832).  Saving model ...
[360/500] train_loss: 1.94135 valid_loss: 1.96383
Validation loss decreased (1.963832 --> 1.963826).  Saving model ...
[361/500] train_loss: 1.94134 valid_loss: 1.9638

[425/500] train_loss: 1.94104 valid_loss: 1.96362
Validation loss decreased (1.963626 --> 1.963623).  Saving model ...
[426/500] train_loss: 1.94104 valid_loss: 1.96362
Validation loss decreased (1.963623 --> 1.963623).  Saving model ...
[427/500] train_loss: 1.94103 valid_loss: 1.96362
Validation loss decreased (1.963623 --> 1.963618).  Saving model ...
[428/500] train_loss: 1.94103 valid_loss: 1.96362
Validation loss decreased (1.963618 --> 1.963616).  Saving model ...
[429/500] train_loss: 1.94102 valid_loss: 1.96362
Validation loss decreased (1.963616 --> 1.963615).  Saving model ...
[430/500] train_loss: 1.94102 valid_loss: 1.96361
Validation loss decreased (1.963615 --> 1.963611).  Saving model ...
[431/500] train_loss: 1.94102 valid_loss: 1.96361
Validation loss decreased (1.963611 --> 1.963609).  Saving model ...
[432/500] train_loss: 1.94101 valid_loss: 1.96360
Validation loss decreased (1.963609 --> 1.963603).  Saving model ...
[433/500] train_loss: 1.94101 valid_loss: 1.9636

In [ ]:
fig, axs = plt.subplots()

sns.lineplot(x=np.arange(1, len(average_training_losses) + 1), y=average_training_losses, ax=axs, label="Training loss")
sns.lineplot(x=np.arange(1, len(average_validation_losses) + 1), y=average_validation_losses, ax=axs, label="Validation loss")
axs.set_xlim((0, None))
plt.tight_layout()

In [ ]:
model = MultiLayerPerceptron(hidden_layer_sizes=(784, ), num_classes=n_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=1e-1)

In [ ]:
patience = 20
num_epochs = 500
model, optimizer, epoch, loss, average_training_losses, average_validation_losses = train_model(
    model, patience, num_epochs, "results/torch_mlp_model.pt", train_loader, validation_loader)

In [ ]:
fig, axs = plt.subplots()

sns.lineplot(x=np.arange(1, len(average_training_losses) + 1), y=average_training_losses, ax=axs, label="Training loss")
sns.lineplot(x=np.arange(1, len(average_validation_losses) + 1), y=average_validation_losses, ax=axs, label="Validation loss")
axs.set_xlim((0, None))
plt.tight_layout()

In [ ]:
model = MultiLayerPerceptron(hidden_layer_sizes=(784, 128, 64, ), num_classes=n_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=1e-1)

In [ ]:
# early stopping patience; how long to wait after last time validation loss improved.
patience = 20
num_epochs = 500
model, optimizer, epoch, loss, average_training_losses, average_validation_losses = train_model(
    model, patience, num_epochs, "results/torch_deep_mlp_model.pt", train_loader, validation_loader)

In [ ]:
fig, axs = plt.subplots()

sns.lineplot(x=np.arange(1, len(average_training_losses) + 1), y=average_training_losses, ax=axs, label="Training loss")
sns.lineplot(x=np.arange(1, len(average_validation_losses) + 1), y=average_validation_losses, ax=axs, label="Validation loss")
axs.set_xlim((0, None))
plt.tight_layout()

In [ ]:
model = ConvolutionalNeuralNetwork(in_channels=1, num_classes=n_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=1e-1)

In [ ]:
# early stopping patience; how long to wait after last time validation loss improved.
patience = 20
num_epochs = 500
model, optimizer, epoch, loss, average_training_losses, average_validation_losses = train_model(
    model, patience, num_epochs, "results/torch_cnn_model.pt", train_loader, validation_loader)

In [ ]:
fig, axs = plt.subplots()

sns.lineplot(x=np.arange(1, len(average_training_losses) + 1), y=average_training_losses, ax=axs, label="Training loss")
sns.lineplot(x=np.arange(1, len(average_validation_losses) + 1), y=average_validation_losses, ax=axs, label="Validation loss")
axs.set_xlim((0, None))
plt.tight_layout()

In [ ]:
model = LSTMModel(input_size=28, hidden_size=100, num_layers=1,
                 bidirectional=False, dropout=0., num_classes=n_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=1e-1)

In [ ]:
# early stopping patience; how long to wait after last time validation loss improved.
patience = 20
num_epochs = 500
model, optimizer, epoch, loss, average_training_losses, average_validation_losses = train_model(
    model, patience, num_epochs, "results/torch_1L_lstm_model.pt", train_loader, validation_loader)

In [ ]:
fig, axs = plt.subplots()

sns.lineplot(x=np.arange(1, len(average_training_losses) + 1), y=average_training_losses, ax=axs, label="Training loss")
sns.lineplot(x=np.arange(1, len(average_validation_losses) + 1), y=average_validation_losses, ax=axs, label="Validation loss")
axs.set_xlim((0, None))
plt.tight_layout()

In [ ]:
model = LSTMModel(input_size=28, hidden_size=100, num_layers=2,
                 bidirectional=False, dropout=0., num_classes=n_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=1e-1)

In [ ]:
# early stopping patience; how long to wait after last time validation loss improved.
patience = 20
num_epochs = 500
model, optimizer, epoch, loss, average_training_losses, average_validation_losses = train_model(
    model, patience, num_epochs, "results/torch_2L_lstm_model.pt", train_loader, validation_loader)

In [ ]:
fig, axs = plt.subplots()

sns.lineplot(x=np.arange(1, len(average_training_losses) + 1), y=average_training_losses, ax=axs, label="Training loss")
sns.lineplot(x=np.arange(1, len(average_validation_losses) + 1), y=average_validation_losses, ax=axs, label="Validation loss")
axs.set_xlim((0, None))
plt.tight_layout()

In [ ]:
model = LSTMModel(input_size=28, hidden_size=100, num_layers=1,
                 bidirectional=True, dropout=0., num_classes=n_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=1e-1)

In [ ]:
# early stopping patience; how long to wait after last time validation loss improved.
patience = 20
num_epochs = 500
model, optimizer, epoch, loss, average_training_losses, average_validation_losses = train_model(
    model, patience, num_epochs, "results/torch_1L_bi_lstm_model.pt", train_loader, validation_loader)

In [ ]:
fig, axs = plt.subplots()

sns.lineplot(x=np.arange(1, len(average_training_losses) + 1), y=average_training_losses, ax=axs, label="Training loss")
sns.lineplot(x=np.arange(1, len(average_validation_losses) + 1), y=average_validation_losses, ax=axs, label="Validation loss")
axs.set_xlim((0, None))
plt.tight_layout()

In [ ]:
model = LSTMModel(input_size=28, hidden_size=100, num_layers=2,
                 bidirectional=True, dropout=0., num_classes=n_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=1e-1)

In [ ]:
# early stopping patience; how long to wait after last time validation loss improved.
patience = 20
num_epochs = 500
model, optimizer, epoch, loss, average_training_losses, average_validation_losses = train_model(
    model, patience, num_epochs, "results/torch_2L_bi_lstm_model.pt", train_loader, validation_loader)

In [ ]:
fig, axs = plt.subplots()

sns.lineplot(x=np.arange(1, len(average_training_losses) + 1), y=average_training_losses, ax=axs, label="Training loss")
sns.lineplot(x=np.arange(1, len(average_validation_losses) + 1), y=average_validation_losses, ax=axs, label="Validation loss")
axs.set_xlim((0, None))
plt.tight_layout()

In [ ]:
checkpoint = torch.load("results/torch_lstm_model.pt")
checkpoint

In [ ]:
model = LSTMModel(input_size=28, hidden_size=100, num_layers=1,
                 bidirectional=False, dropout=0., num_classes=n_classes)
optimizer = optim.SGD(model.parameters(), lr=1e-1)

model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
# loss = checkpoint['validation_loss']

model.eval()

In [ ]:
model.eval()
y_true = []
y_pred = []

with torch.no_grad():
    for inputs, targets in tqdm(train_loader):
        inputs = inputs.to(device)
        targets = targets.to(device)
        outputs = model(inputs)
        if task == 'multi-label, binary-class':
            targets = targets.to(torch.float32)
        else:
            targets = targets.squeeze().long()
            targets = targets.float().resize_(len(targets), 1)

        y_true.append(targets.detach().numpy().flatten())
        y_pred.append(outputs.detach().numpy().argmax(axis=1))
accuracy_score(np.hstack(y_true), np.hstack(y_pred))

In [ ]:
model.eval()
y_true = []
y_pred = []

with torch.no_grad():
    for inputs, targets in tqdm(test_loader):
        inputs = inputs.to(device)
        targets = targets.to(device)
        outputs = model(inputs)
        if task == 'multi-label, binary-class':
            targets = targets.to(torch.float32)
        else:
            targets = targets.squeeze().long()
            targets = targets.float().resize_(len(targets), 1)

        y_true.append(targets.detach().numpy().flatten())
        y_pred.append(outputs.detach().numpy().argmax(axis=1))
accuracy_score(np.hstack(y_true), np.hstack(y_pred))

# We then check a 3D dataset

In [ ]:
data_flag = 'organmnist3d'
download = True

info = INFO[data_flag]
DataClass = getattr(medmnist, info['python_class'])

# load the data
train_dataset = DataClass(split='train',  download=download)

# encapsulate data into dataloader form
train_loader = data.DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [ ]:
x, y = train_dataset[0]

print(x.shape, y.shape)

In [ ]:
for x, y in train_loader:
    print(x.shape, y.shape)
    break

In [ ]:
frames = train_dataset.montage(length=1, save_folder="tmp/")
frames[10]

In [ ]:
frames = train_dataset.montage(length=20, save_folder="tmp/")

frames[10]